# Writing Indicators

This notebook covers implementing a custom stock indicator in **PyBroker** and integrating it into a trading strategy.

**PyBroker** uses vectorized indicators that are computed on all bars of data at a single time. This makes [NumPy](https://numpy.org/) a natural choice for writing indicators in **PyBroker**:

In [1]:
import numpy as np
from numba import njit

We also import [Numba](https://numba.pydata.org/), a Just In Time (JIT) compiler that translates a subset of Python into fast machine code. Numba is ideal for code that works with loops and NumPy arrays.  

Below is an indicator function that calculates close prices minus a moving average (CMMA), which could be used for a [mean reversion](https://en.wikipedia.org/wiki/Mean_reversion_(finance)) strategy:

In [2]:
def cmma(bar_data, lookback):
    @njit  # Enable Numba JIT.
    def vec_cmma(values):
        # Initialize the result array.
        n = len(values)
        out = np.array([np.nan for _ in range(n)])
        
        # For all bars starting at lookback:
        for i in range(lookback, n):
            # Calculate the moving average for the lookback.
            ma = 0
            for j in range(i - lookback, i):
                ma += values[j]
            ma /= lookback
            # Subtract the moving average from value.
            out[i] = values[i] - ma
        return out
    
    # Calculate with close prices.
    return vec_cmma(bar_data.close)

The ```cmma``` function takes two arguments. The first argument, ```bar_data```, is an instance of [BarData](https://www.pybroker.com/en/latest/reference/pybroker.common.html#pybroker.common.BarData). The BarData class holds fields for open, high, low, and close prices (OHLC), as well as all custom fields that were registered with **PyBroker**. The second argument, ```lookback```, is a user-defined argument for the lookback of the moving average.

Take note of the nesting of the ```vec_cmma``` function. Nesting is needed since ```vec_cmma``` is JIT-compiled by Numba. A Numba compiled function will support a NumPy array as an argument, but not an instance of a Python class like ```BarData```.

As mentioned, indicators in **PyBroker** are *vectorized*. The indicator values are computed over all of the historical data at one time, as shown in ```vec_cmma```. Making this computation all at once makes backtests run much faster!

Next, the indicator function is registered with **PyBroker**:

In [3]:
import pybroker

ind_cmma_20 = pybroker.indicator('cmma_20', cmma, lookback=20)

The above registers the indicator function with **PyBroker** using a name of ```cmma_20```, denoting the ```lookback``` of 20 bars. Any arguments to the indicator function that appear after ```bar_data``` are treated as user-defined arguments that are passed to [pybroker.indicator](https://www.pybroker.com/en/latest/reference/pybroker.indicator.html#pybroker.indicator.indicator). After the indicator function is registered with **PyBroker**, a new [Indicator](https://www.pybroker.com/en/latest/reference/pybroker.indicator.html#pybroker.indicator.Indicator) instance is returned that references the indicator function we defined.

Now, let us test the ```Indicator``` with some data downloaded from [Yahoo Finance](https://finance.yahoo.com):

In [4]:
from pybroker import YFinance

pybroker.enable_data_source_cache('yfinance')

yfinance = YFinance()
df = yfinance.query('PG', '4/1/2020', '4/1/2022')

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:02 



In [5]:
ind_cmma_20(df)

2020-04-01 04:00:00         NaN
2020-04-02 04:00:00         NaN
2020-04-03 04:00:00         NaN
2020-04-06 04:00:00         NaN
2020-04-07 04:00:00         NaN
                         ...   
2022-03-25 04:00:00    1.967502
2022-03-28 04:00:00    3.288005
2022-03-29 04:00:00    4.968507
2022-03-30 04:00:00    3.790999
2022-03-31 04:00:00    2.171002
Length: 505, dtype: float64

As you can see, the ```Indicator``` instance is a ```Callable```. Once called, a [Pandas Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) containing the computed indicator values is returned.

The ```Indicator``` class also contains functions for measuring its information content, like its interquartile range (IQR):

In [6]:
ind_cmma_20.iqr(df)

4.655495452880842

And its relative entropy:

In [7]:
ind_cmma_20.relative_entropy(df)

0.7495800114455111

## Using the Indicator in a Strategy

Once we are satisified with our indicator's implementation, the next step is to use it in a trading strategy. The simple strategy below goes long when the 20-day CMMA is less than 0 – i.e. when the last close price drops below the 20-day moving average:

In [8]:
def buy_cmma_cross(ctx):
    if ctx.long_pos():
        return
    # Place a buy order if the most recent value of the 20 day CMMA is < 0:
    if ctx.indicator('cmma_20')[-1] < 0:
        ctx.buy_shares = ctx.calc_target_shares(1)
        ctx.hold_bars = 3

The indicator values are retrieved by calling [ctx.indicator](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.BaseContext.indicator) on the [ExecContext](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext) and passing in the ```cmma_20``` indicator name that was registered.

In [9]:
from pybroker import Strategy

strategy = Strategy(yfinance, '4/1/2020', '4/1/2022')
strategy.add_execution(buy_cmma_cross, 'PG', indicators=ind_cmma_20)

The ```buy_cmma_cross``` function was added to the [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) along with the ```ind_cmma_20``` ```Indicator```.

In [10]:
pybroker.enable_indicator_cache('my_indicators')

**PyBroker** also allows caching the computed indicator values to disk.

In [11]:
result = strategy.backtest(calc_bootstrap=False)
result.metrics_df

Backtesting: 2020-04-01 00:00:00 to 2022-04-01 00:00:00

Loaded cached bar data.

Generating indicators...


100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



Test split: 2020-04-01 04:00:00 to 2022-03-31 04:00:00


100% (505 of 505) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:01


,name,value
0,trade_count,60.000000
1,initial_market_value,100000.000000
2,end_market_value,100759.360000
3,total_pnl,759.360000
4,total_return_pct,0.759360
5,total_profit,41596.750000
6,total_loss,-40837.390000
7,total_fees,0.000000
8,max_drawdown,-13446.930000
9,max_drawdown_pct,-11.977356


When the backtest runs, **PyBroker** computes the indicator values. If there were multiple indicators added to the ```Strategy```, then **PyBroker** will compute them in parallel across multiple CPU cores.

## Vectorized Helpers

**PyBroker** includes several vectorized helper functions. One of them is [highv](https://www.pybroker.com/en/latest/reference/pybroker.vect.html#pybroker.vect.highv), which computes the highest value for every preceding ```lookback``` period.

Below defines an indicator that computes the "highest" high price for every period of 5 bars.

In [12]:
from pybroker.vect import highv

def hhv(bar_data, lookback):
    return highv(bar_data.high, lookback)

ind_hhv_5 = pybroker.indicator('hhv_5', hhv, lookback=5)

In [13]:
ind_hhv_5(df)

2020-04-01 04:00:00           NaN
2020-04-02 04:00:00           NaN
2020-04-03 04:00:00           NaN
2020-04-06 04:00:00           NaN
2020-04-07 04:00:00    120.059998
                          ...    
2022-03-25 04:00:00    153.919998
2022-03-28 04:00:00    153.919998
2022-03-29 04:00:00    156.470001
2022-03-30 04:00:00    156.470001
2022-03-31 04:00:00    156.470001
Length: 505, dtype: float64

The [pybroker.vect](https://www.pybroker.com/en/latest/reference/pybroker.vect.html) module also includes other vectorized helpers such as [lowv](https://www.pybroker.com/en/latest/reference/pybroker.vect.html#pybroker.vect.lowv), [sumv](https://www.pybroker.com/en/latest/reference/pybroker.vect.html#pybroker.vect.sumv), and [cross](https://www.pybroker.com/en/latest/reference/pybroker.vect.html#pybroker.vect.cross), the last of which is used to compute crossovers.

[The next step is to learn how to train a model using our custom indicator.](https://www.pybroker.com/en/latest/notebooks/6.%20Training%20a%20Model%20with%20Walkforward%20Analysis.html)